In [41]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras import backend as K
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau,EarlyStopping
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import GridSearchCV

def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1))
def strd(s):
    return (s-np.mean(s))/np.std(s)

In [2]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

def drop_vars(df):
    tmp=df.shape[1]
    df = df[df.columns[[True]+list((df.var()!=0))]]
    print('0 var:',tmp-df.shape[1])
    
    corr_matrix = df[df.columns[2:]].corr().abs()
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
    to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]
    tmp=df.shape[1]
    df=df.drop(to_drop, axis=1)
    print('Corr>0.95:',tmp-df.shape[1],'Now:',)
    
    corrs = dict()
    for i in range(df.shape[1]-2):
        corrs[df.columns[2+i]] = np.corrcoef(df['target'],df[df.columns[2+i]])[0,1]
    s = [k for k in corrs if abs(corrs[k])<0.1]
    tmp=df.shape[1]
    df=df.drop(s, axis=1)
    print('Corr Target <0.1:',tmp-df.shape[1])
    
    return df
train = drop_vars(train)

0 var: 256
Corr>0.95: 139 Now:
Corr Target <0.1: 4206


## Test Improvement

In [46]:
X=train[train.columns[2:]]
y=np.log1p(train['target'])

x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=203)

sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)

clf = GradientBoostingRegressor()
clf.fit(x_train, y_train)

y_true,y_pred = y_test,clf.predict(x_test) 
print(root_mean_squared_error(y_pred, y_true))

xtrainpred = strd(clf.predict(x_train).flatten().reshape(-1,1))
xtestpred = strd(clf.predict(x_test).flatten().reshape(-1,1))
x_train = np.c_[x_train,xtrainpred]
x_test = np.c_[x_test,xtestpred]

model = Sequential()
model.add(Dense(50, input_dim=x_train.shape[1], activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1))

model.compile(loss=root_mean_squared_error,
              optimizer=Adam(lr=0.1,decay=0.0001))


checkp = ModelCheckpoint(filepath='weights.hdf5')
lrred = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=10, cooldown=2, min_lr=0.000001)
stp = EarlyStopping(monitor='val_loss', min_delta=0, patience=50)
cbs = [checkp,lrred,stp]
model.fit(x_train, y_train,
        epochs=1000,
        batch_size=400,
        validation_data=(x_test, y_test),
        callbacks=cbs)

Tensor("Sqrt_2:0", shape=(), dtype=float64)
Train on 3567 samples, validate on 892 samples
Epoch 1/1000
3567/3567 [==============================] - 0s 105us/step - loss: 14.6225 - val_loss: 7.3356
Epoch 2/1000
3567/3567 [==============================] - 0s 15us/step - loss: 9.3249 - val_loss: 8.3023
Epoch 3/1000
3567/3567 [==============================] - 0s 16us/step - loss: 8.4927 - val_loss: 5.8363
Epoch 4/1000
3567/3567 [==============================] - 0s 15us/step - loss: 7.6271 - val_loss: 5.5776
Epoch 5/1000
3567/3567 [==============================] - 0s 15us/step - loss: 6.6156 - val_loss: 5.2309
Epoch 6/1000
3567/3567 [==============================] - 0s 15us/step - loss: 5.9183 - val_loss: 5.7721
Epoch 7/1000
3567/3567 [==============================] - 0s 15us/step - loss: 6.1921 - val_loss: 6.3851
Epoch 8/1000
3567/3567 [==============================] - 0s 15us/step - loss: 5.5374 - val_loss: 4.5668
Epoch 9/1000
3567/3567 [==============================] - 0s 15us/s

3567/3567 [==============================] - 0s 15us/step - loss: 1.2079 - val_loss: 1.2998
Epoch 78/1000
3567/3567 [==============================] - 0s 15us/step - loss: 1.2250 - val_loss: 1.3104
Epoch 79/1000
3567/3567 [==============================] - 0s 16us/step - loss: 1.2246 - val_loss: 1.3136
Epoch 80/1000
3567/3567 [==============================] - 0s 16us/step - loss: 1.2249 - val_loss: 1.3178
Epoch 81/1000
3567/3567 [==============================] - 0s 16us/step - loss: 1.2223 - val_loss: 1.3120
Epoch 82/1000
3567/3567 [==============================] - 0s 15us/step - loss: 1.2166 - val_loss: 1.3186
Epoch 83/1000
3567/3567 [==============================] - 0s 15us/step - loss: 1.2129 - val_loss: 1.3089
Epoch 84/1000
3567/3567 [==============================] - 0s 15us/step - loss: 1.2265 - val_loss: 1.3076
Epoch 85/1000
3567/3567 [==============================] - 0s 15us/step - loss: 1.2175 - val_loss: 1.3134
Epoch 86/1000
3567/3567 [==============================] - 0

## NN+ElNet - Actual

In [ ]:
x_train=train[train.columns[2:]]
y_train=np.log1p(train['target'])

x_test=test[x_train.columns]

sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)

clf = GradientBoostingRegressor()
clf.fit(x_train, y_train)
#y_true,y_pred = y_test,clf.predict(x_test) 
#print(root_mean_squared_error(y_pred, y_true))

xtrainpred = strd(clf.predict(x_train).flatten().reshape(-1,1))
xtestpred = strd(clf.predict(x_test).flatten().reshape(-1,1))
x_train = np.c_[x_train,xtrainpred]
x_test = np.c_[x_test,xtestpred]

model = Sequential()
model.add(Dense(50, input_dim=x_train.shape[1], activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1))

model.compile(loss=root_mean_squared_error,
              optimizer=Adam(lr=0.1,decay=0.0001))


checkp = ModelCheckpoint(filepath='weights.hdf5')
lrred = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=10, cooldown=2, min_lr=0.000001)
stp = EarlyStopping(monitor='loss', min_delta=0, patience=50)
cbs = [checkp,lrred,stp]
model.fit(x_train, y_train,
        epochs=1000,
        batch_size=400,
        callbacks=cbs)

Epoch 1/1000
4459/4459 [==============================] - 0s 81us/step - loss: 16.4367
Epoch 2/1000
4459/4459 [==============================] - 0s 15us/step - loss: 9.8158
Epoch 3/1000
4459/4459 [==============================] - 0s 15us/step - loss: 8.1261
Epoch 4/1000
4459/4459 [==============================] - 0s 15us/step - loss: 6.9351
Epoch 5/1000
4459/4459 [==============================] - 0s 15us/step - loss: 6.2995
Epoch 6/1000
4459/4459 [==============================] - 0s 14us/step - loss: 5.5920
Epoch 7/1000
4459/4459 [==============================] - 0s 14us/step - loss: 4.7647
Epoch 8/1000
4459/4459 [==============================] - 0s 15us/step - loss: 4.1553
Epoch 9/1000
4459/4459 [==============================] - 0s 14us/step - loss: 3.6457
Epoch 10/1000
4459/4459 [==============================] - 0s 14us/step - loss: 3.1106
Epoch 11/1000
4459/4459 [==============================] - 0s 14us/step - loss: 2.8403
Epoch 12/1000
4459/4459 [==========================

4459/4459 [==============================] - 0s 14us/step - loss: 1.1857
Epoch 189/1000
4459/4459 [==============================] - 0s 14us/step - loss: 1.1970
Epoch 190/1000
4459/4459 [==============================] - 0s 15us/step - loss: 1.1850
Epoch 191/1000
4459/4459 [==============================] - 0s 13us/step - loss: 1.2009
Epoch 192/1000
4459/4459 [==============================] - 0s 14us/step - loss: 1.1973
Epoch 193/1000
4459/4459 [==============================] - 0s 13us/step - loss: 1.1890
Epoch 194/1000
4459/4459 [==============================] - 0s 13us/step - loss: 1.1899
Epoch 195/1000
4459/4459 [==============================] - 0s 14us/step - loss: 1.1865
Epoch 196/1000
4459/4459 [==============================] - 0s 14us/step - loss: 1.1790
Epoch 197/1000
4459/4459 [==============================] - 0s 15us/step - loss: 1.1863
Epoch 198/1000
4459/4459 [==============================] - 0s 14us/step - loss: 1.1998
Epoch 199/1000
4459/4459 [=====================

In [18]:
predictions=pd.DataFrame({'ID':test['ID'],'target':np.expm1(model.predict(x_test, verbose=1).flatten())})
print(predictions.head())
predictions.to_csv('pred_boost_el.csv',index=False)

49342/49342 [==============================] - 1s 15us/step
          ID      target
0  000137c73  2407030.75
1  00021489f  2235604.75
2  0004d7953  3764524.50
3  00056a333  2877247.50
4  00056d8eb  2235604.75
